# File format conversion
CMG sr3/gmch.sr3 --> CMG rwo (by running CMG Results Report software on rwd files) --> numpy array

# convert CMG sr3 file to rwo files

sr3 is the CMG simulation result file, whereas rwo is the extracted info in the ascii format

In [ ]:
from pathlib import Path
from CMG2npy import generate_CMG_rwd, run_CMG_rwd_report

base_path = Path('.')
folder_path = base_path/"data"
property = 'STRESMXP'
for case_num in range(1, 3):
    generate_CMG_rwd(
        sr3_folder_path = folder_path,
        case_name = f'case{case_num}',
        property = property,
        is_gmc_property = True,
        precision = 4
    )

    run_CMG_rwd_report(
        rwd_folder_path = folder_path,
        case_name = f'case{case_num}',
        cmg_version = 'ese-ts2win-v2024.20',
    )

    # remove rwd files
    rwd_folder_path = folder_path
    Path(rwd_folder_path, f'case{case_num}.rwd').unlink()

case1: writing rwd file ...
case1: generating rwo file ...
case2: writing rwd file ...
case2: generating rwo file ...


sh: C:\Program Files\CMG\RESULTS\2024.20\Win_x64\exe\Report.exe: command not found
sh: C:\Program Files\CMG\RESULTS\2024.20\Win_x64\exe\Report.exe: command not found


# extract simulation results from rwo files into numpy arrays

## option 1: extract results on all grid cells

In [ ]:
from pathlib import Path
from CMG2npy import CMG_rwo2npy

base_path = Path('.')
for case_num in range(1, 3):
    sim_results = CMG_rwo2npy(
        rwo_folder_path = base_path/'data'/'rwo',
        case_name = f'case{case_num}',
        property = property,
        is_save = True,
        save_folder_path = base_path/'results',
        show_info = False
    )

## option 2: only keep fault grid cells and simulated layers

In [ ]:
import numpy as np
from pathlib import Path
from CMG2npy import CMG_rwo2npy

# load the grid containing fault id
coor_fault = np.load('results/JD_Sula_2025_gmc_coor&fault.npy')
base_path = Path('.')
save_folder_path = base_path/"results"
save_folder_path.mkdir(exist_ok=True)

for case_num in range(1, 3):
    # extract results on all grid cells
    sim_results = CMG_rwo2npy(
        rwo_folder_path = base_path/'data'/'rwo',
        case_name = f'case{case_num}',
        property = property,
        is_save = False,
        save_folder_path = base_path/'results',
        show_info = False
    )

    # keep results on fault grid cells only
    nan_mask = np.isnan(coor_fault[:,:,:,3])
    sim_results[nan_mask] = np.nan

    # keep results of simulated layers only
    # Python indices are 0-based, to extract slices 41–79, use 40:79
    sim_results_trimmed = sim_results[:, :, 5:10]

    # save
    np.save(save_folder_path/f'case{case_num}_{property}.npy')

Parsing file: data/case1_PRES.rwo
Processing time step: 0.0 (2030-Jan-01)
Processing time step: 3652.0 (2040-Jan-01)
Processing time step: 7305.0 (2050-Jan-01)
Processing time step: 10957.0 (2060-Jan-01)
Processing time step: 189926.0 (2550-Jan-01)
Processing time step: 372547.0 (3050-Jan-01)
Found 6 time steps
Grid dimensions: I=107, J=117, K=79, Time=6


In [13]:
import numpy as np

my_array  = np.load('results/JD_Sula_2025_flow_coor&fault.npy')
my_array = my_array [...,3]
# Calculate basic statistics of a array
print('Shape: ', my_array.shape)
print(f"Min: {np.min(my_array):.2f}")
print(f"Max: {np.max(my_array):.2f}")
print(f"Sum: {np.sum(my_array):.2f}")
print(f"Mean: {np.mean(my_array):.2f}")
print(f"Median: {np.median(my_array)}")
print(f"Std: {np.std(my_array):.2f}")

Shape:  (107, 117, 79)
Min: nan
Max: nan
Sum: nan
Mean: nan
Median: nan
Std: nan


In [8]:
import numpy as np

my_array  = np.load('results/JD_Sula_2025_flow_coor&fault.npy')
my_array = my_array [...,0]
# Calculate basic statistics of a array
print('Shape: ', my_array.shape)
print(f"Min: {np.nanmin(my_array):.2f}")
print(f"Max: {np.nanmax(my_array):.2f}")
print(f"Sum: {np.nansum(my_array):.2f}")
print(f"Mean: {np.nanmean(my_array):.2f}")
print(f"Median: {np.nanmedian(my_array)}")
print(f"Std: {np.nanstd(my_array):.2f}")

Shape:  (107, 117, 79)
Min: 424730.20
Max: 477843.57
Sum: 446336817127.22
Mean: 451300.67
Median: 451303.64249999996
Std: 15327.59


In [9]:
my_array[0,0,0]

425220.4375

## keep results of simulated layers only

for the JD_Sula_2025_flow grid (107x117x79), the reservoir layers are k=41-79. The other layers were not simulated by setting ntg=0.

In [ ]:
import os
import numpy as np

# Input and output folders
input_folder = "data"             
output_folder = "results_trimmed"    
os.makedirs(output_folder, exist_ok=True)

# Loop through files
for filename in os.listdir(input_folder):
    if filename.endswith("_PRES.npy"):
        input_path = os.path.join(input_folder, filename)

        # Load array
        arr = np.load(input_path) 

        # Extract slices 41–79 (Python indices are 0-based, so use 40:79)
        arr_trimmed = arr[:, :, 40:79]   # shape (107,117,39,6)

        # Save to output folder with same name
        output_path = os.path.join(output_folder, filename)
        np.save(output_path, arr_trimmed)

        print(f"Processed {filename} -> {arr_trimmed.shape}")
